In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import json
import requests

In [2]:
ranking_url = 'https://itunes.apple.com/jp/rss/topfreeapplications/limit=100/json'
raw_ranking = requests.get(ranking_url).json()
ranking = [{'id': entry['id']['attributes']['im:id'], 'name': entry['im:name']}for entry in raw_ranking['feed']['entry']]

In [3]:
data = []
for element in ranking:
    review_url = 'https://itunes.apple.com/jp/rss/customerreviews/id={0}/json'.format(element['id'])
    raw_reviews = requests.get(review_url).json()
    try:
        reviews = [review['content']['label'] for review in raw_reviews['feed']['entry'][1:50]]
        for review in reviews:
            data.append({'id': element['id'], 'name': element['name'], 'review': review.encode('utf-8')})
    except KeyError:
        continue
data = Series(data)
data = data.dropna()

In [11]:
data.to_csv("review.csv", encoding="UTF-8")

In [4]:
from AI import AI
ai = AI()

In [5]:
#AIにドキュメントを登録
result = []
for i in range(len(data)):
    result.append(ai.register_document(i, 889, data[i]['review']))

In [10]:
#みんながスパムと判断したやつを突っ込む
h = 3159
hoge = ["♪( ´▽｀)", "あ", "なかなか", "wcuigt", "い", "00ba50使って下さい", "28B0D0C162", "たの", "Love the Naruto gacha toy....Can't wait for Misuji steak....simply deliious"]
for i in range(len(hoge)):
    ai.register_document(i+h, 889, hoge[i])

In [12]:
# 教師データ登録
ai.register_teacher(1, 889, [3159, 3160, 3161, 3162, 3163, 3164, 3165, 3166, 3167], )

u'{"result":"success"}'

In [24]:
ai.register_teacher(2, 889, [3159, 3160, 3161, 3162, 3163, 3164, 3165, 3166, 3167], [1, 4, 33, 44, 60, 61, 80, 114, 148, 174])

u'{"result":"success"}'

In [37]:
spam_ids = [i['documentId'] for i in json.loads(ai.get_result(2, 889))['documents']][0:8]

In [42]:
spam_ids = [2504, 207, 210, 1383, 1400, 3101]

In [45]:
#スパム削除
data = data.drop(spam_ids)

In [46]:
samples = np.random.randint(len(data), size=(100))

In [47]:
samples = data[samples]

In [48]:
samples.to_csv('sample4.csv', encoding="UTF-8")

In [53]:
tomobe = pd.read_csv("sample2/tomobe.csv", header=None)

In [57]:
tomobe = tomobe[[0, 2, 3]]

In [74]:
positiv = data[tomobe[tomobe[2] == True][0]]

In [81]:
positiv.to_csv('positiv.csv', encoding="UTF-8")

In [94]:
is_positiv = (tomobe[2] == 1)
is_not_positiv = ~is_positiv

In [102]:
positiv_tomobe_id = []
for i in tomobe[is_positiv][0].values:
    positiv_tomobe_id.append(i)

In [104]:
not_positiv_tomobe_id = []
for i in tomobe[is_not_positiv][0].values:
    not_positiv_tomobe_id.append(i)

In [165]:
result = []
for i in data.index:
    result.append(ai.register_document(i, 887, data[i]['review']))

In [185]:
positiv_tomobe_id.remove(1140)

In [193]:
ai.register_teacher(11, 887, positiv_tomobe_id[0:11], not_positiv_tomobe_id[0:11])

u'{"result":"success"}'

In [339]:
#ここまで 14:30
positiv_result = json.loads(ai.get_result(11, 887))['documents']

In [310]:
#ここから15:00
ai.register_teacher(12,887, not_positiv_tomobe_id[0:11], positiv_tomobe_id[0:11])

u'{"result":"success"}'

In [341]:
negativ_result = json.loads(ai.get_result(12, 887))['documents']

In [355]:
negativ_result[0]

{u'documentId': 2781, u'score': 4038}

In [356]:
for nr in negativ_result:
    data[nr['documentId']]['negative_result'] = nr['score']

In [379]:
data[1]

{'id': u'443904275',
 'name': {u'label': u'LINE'},
 'negative_result': 0,
 'positive_score': 0,
 'review': '\xe3\x82\x84\xe3\x81\xa3\xe3\x81\xb1\xe3\x82\x8a\xe3\x81\x93\xe3\x82\x8c\xe3\x81\x8c\xe3\x81\xaa\xe3\x81\x84\xe3\x81\xa8\xe3\x81\xa0\xe3\x82\x81\xe3\x80\x82\n\xe3\x81\x97\xe3\x81\x84\xe3\x81\xa6\xe6\x82\xaa\xe3\x81\x84\xe3\x81\xa8\xe3\x81\x93\xe3\x82\x8d\xe3\x82\x92\xe3\x81\x82\xe3\x81\x92\xe3\x82\x8b\xe3\x81\xaa\xe3\x82\x89\xe3\x80\x81\xe5\x8f\x8b\xe9\x81\x94\xe8\xbf\xbd\xe5\x8a\xa0\xe3\x81\x97\xe3\x81\x9f\xe3\x81\xa8\xe3\x81\x8d\xe3\x81\xab\xe9\x87\x8d\xe3\x81\x84\xe3\x81\x8b\xe3\x82\x89\xe3\x81\x8b\xe3\x81\x97\xe3\x82\x89\xe3\x82\x93\xe3\x81\x91\xe3\x81\xa9\xe3\x81\x8a\xe3\x81\xa1\xe3\x82\x8b\xe3\x80\x82\xe3\x81\x9d\xe3\x81\x93\xe3\x81\x8c\xe6\x9c\x80\xe6\x82\xaa'}

In [380]:
tmp_score = {}
for di in data.index[1:]:
    if tmp_score.has_key(data[di]['id']):
       tmp_score[data[di]['id']]['count'] += 1
       tmp_score[data[di]['id']]['score'] += data[di]['positive_score'] - data[di]['negative_result']
    else:
        tmp_score[data[di]['id']] = {'score': data[di]['positive_score'] - data[di]['negative_result'], 'count': 1}

In [390]:
result_score = {}
for k, v in tmp_score.items():
    result_score[k] = v['score']/v['count']

In [396]:
semifinal_result = sorted(result_score.items(), key=lambda x:x[1], reverse=True)

In [397]:
semifinal_result

[(u'389801252', 77),
 (u'984440479', 62),
 (u'876258957', 37),
 (u'955367905', 17),
 (u'980865528', 14),
 (u'952608759', 8),
 (u'983904653', -1),
 (u'446278045', -5),
 (u'958048961', -6),
 (u'959454698', -15),
 (u'284815942', -17),
 (u'607974779', -19),
 (u'600843984', -21),
 (u'876291194', -27),
 (u'946755072', -30),
 (u'696463126', -37),
 (u'985072363', -37),
 (u'472143590', -46),
 (u'658511662', -48),
 (u'721512660', -69),
 (u'909566506', -69),
 (u'931854667', -86),
 (u'551682016', -87),
 (u'933166032', -90),
 (u'949244365', -91),
 (u'579581125', -99),
 (u'850417475', -99),
 (u'667861049', -105),
 (u'447339142', -105),
 (u'633246396', -105),
 (u'544007664', -116),
 (u'516561342', -122),
 (u'374254473', -122),
 (u'963073142', -140),
 (u'491903216', -148),
 (u'291676451', -157),
 (u'956081467', -158),
 (u'970817084', -165),
 (u'437758919', -166),
 (u'919478091', -171),
 (u'454638411', -172),
 (u'952578897', -172),
 (u'905173595', -175),
 (u'673540001', -175),
 (u'714796093', -177),
 (

In [465]:
final_result = []
lookup_url = 'https://itunes.apple.com/jp/app/id'
for i in semifinal_result:
    lookup_result = requests.get(lookup_url + i[0].encode('utf8')).text
    d = pq(lookup_result)
    img = d("#left-stack > div.lockup.product.application > a:nth-child(1) > div > img")
    image_url = img.attr("src-swap-high-dpi")
    final_result.append({'id': i[0], 'score': i[1], 'image': image_url})

In [468]:
json.dumps(final_result)

'[{"image": "http://a2.mzstatic.com/jp/r30/Purple1/v4/da/96/ab/da96ab96-8699-495d-4af6-d8215662d449/icon350x350.png", "score": 77, "id": "389801252"}, {"image": "http://a1.mzstatic.com/jp/r30/Purple7/v4/86/3f/67/863f67c3-22d0-6e87-62e3-a96d58ed60c6/icon350x350.png", "score": 62, "id": "984440479"}, {"image": "http://a3.mzstatic.com/jp/r30/Purple5/v4/a0/55/88/a0558837-c1a1-2da2-a082-a4afb7482f72/icon350x350.png", "score": 37, "id": "876258957"}, {"image": "http://a1.mzstatic.com/jp/r30/Purple3/v4/9c/d0/ac/9cd0ac97-229d-623e-d8fa-606ef2d3f6ed/icon350x350.png", "score": 17, "id": "955367905"}, {"image": "http://a3.mzstatic.com/jp/r30/Purple1/v4/b7/a4/de/b7a4deaf-3639-0508-5ff6-afcd898aad77/icon350x350.jpeg", "score": 14, "id": "980865528"}, {"image": "http://a2.mzstatic.com/jp/r30/Purple5/v4/91/79/f4/9179f483-7b8d-2a45-8daa-caee9ac5852a/icon350x350.jpeg", "score": 8, "id": "952608759"}, {"image": "http://a4.mzstatic.com/jp/r30/Purple7/v4/1b/98/c4/1b98c4d5-96ff-f85c-d2ac-977f275bf09b/icon3

In [419]:
response = requests.get('https://itunes.apple.com/lookup?id=876258957').json()
response['results'][0]['artworkUrl512']

u'http://is5.mzstatic.com/image/pf/us/r30/Purple3/v4/12/17/2c/12172ca4-b044-b56a-7e7e-493480558b6c/pr_source.png'

In [398]:
result = []
lookup_url = 'https://itunes.apple.com/jp/app/id'

In [399]:
lookup_result = requests.get(lookup_url + '981863874')

In [400]:
from pyquery import PyQuery as pq

In [218]:
#left-stack > div.lockup.product.application > a:nth-child(1) > div > img

<Response [200]>

In [463]:
lookup_result = requests.get('https://itunes.apple.com/jp/app/id876258957')
d = pq(lookup_result.text)
img = d("#left-stack > div.lockup.product.application > a:nth-child(1) > div > img")
img.attr("src-swap-high-dpi")

'http://a3.mzstatic.com/jp/r30/Purple5/v4/a0/55/88/a0558837-c1a1-2da2-a082-a4afb7482f72/icon350x350.png'

In [537]:
len(semifinal_result)

65

In [489]:
import copy

In [514]:
hoge = copy.deepcopy(final_result)

In [516]:
for i in final_result:
    i['positive'] = ""
    i['negative'] = ""

In [519]:
positiv_result[0:1]

[{u'documentId': 182, u'score': 2798}]

In [534]:
for i in positiv_result:        
    id = data[i['documentId']]['id']
    review = data[i['documentId']]['review']
    for j in final_result:
        if j['id'] == id:
           j['positive'] = review

In [576]:
for i in negativ_result:        
    id = data[i['documentId']]['id']
    review = data[i['documentId']]['review']
    for j in final_result:
        if j['id'] == id:
           if j['positive'] != review:
              j['negative'] = review

In [578]:
import codecs
f = codecs.open('result.json', 'w', 'utf-8')
f.write(json.dumps(final_result))
f.close()

In [591]:
print data[positiv_result[8]['documentId']]['review']

1日で全クリしちゃったし、クリスタルも全部集めた。新しいステージ出来るの遅すぎ…
あんま新ステージ出来るの遅いともう辞めちゃうかもー


In [601]:
print data[negativ_result[0]['documentId']]['review']

他のアプリから戻ったあと強制的にトップに戻るのは非常に良くない。記事に関する調べごともおちおちできない。強制的にApp Storeに飛ぶリンク記事仕込まれててヤフーニュースに戻ったら読んでいた記事ではなくトップに強制移動とかかなりひどいよねこれ。
